In [1]:
#download repo
!git clone https://github.com/victorchall/EveryDream.git
# Set working directory
%cd EveryDream

c:\Users\texas\Desktop\Personal\SceneFusion\AnimateDiff-SceneFusion\data\EveryDream


Cloning into 'EveryDream'...
C:\Users\texas\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [10]:

# install requirements
# !pip install torch=='1.12.1+cu113' 'torchvision==0.13.1+cu113' --extra-index-url https://download.pytorch.org/whl/cu113
!pip install pandas>='1.3.5'
!git clone https://github.com/salesforce/BLIP scripts/BLIP
!pip install timm
!pip install fairscale
# !pip install transformers=='4.19.2'
!pip install timm
!pip install aiofiles
!pip install aiohttp


fatal: destination path 'scripts/BLIP' already exists and is not an empty directory.


     ---------------------------------------- 0.0/266.3 kB ? eta -:--:--
     - -------------------------------------- 10.2/266.3 kB ? eta -:--:--
     ---- -------------------------------- 30.7/266.3 kB 325.1 kB/s eta 0:00:01
     --------------------------- ---------- 194.6/266.3 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 266.3/266.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for fairscale: filename=fairscale-0.4.13-py3-none-any.whl size=332117 sha256=5b1df0e5e9058fefe2a3ecb3a03ac701ccac1518886122935326cf1eab31ea14
  Stored in directo

In [11]:
!python scripts/auto_caption.py --img_dir raw/1 --out_dir output 

Windows detected, using asyncio.WindowsSelectorEventLoopPolicy
starting
input_dir:  raw/1
Model cached to: .cache/model_base_caption_capfilt_large.pth
load checkpoint from .cache/model_base_caption_capfilt_large.pth
loading model to cuda



(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]
(…)bert-base-uncased/resolve/main/vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 28.9MB/s]

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]
(…)cased/resolve/main/tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]
(…)rt-base-uncased/resolve/main/config.json: 100%|██████████| 570/570 [00:00<00:00, 569kB/s]
Traceback (most recent call last):
  File "c:\Users\texas\Desktop\Personal\SceneFusion\AnimateDiff-SceneFusion\data\EveryDream\scripts\auto_caption.py", line 217, in <module>
    asyncio.run(main(opt))
  File "c:\Users\texas\anaconda3\envs\animatediff\lib\asyncio\runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "c:\Users\texas\anaconda3\envs\animatediff\lib\asyncio\base_events.py", line 649, in run_until_complete
    return f